The code was adapted from the tutorial 'https://naturalistic-data.org/content/Intersubject_Correlation.html', which was written by Juha Lahnakoski and Luke Chang

In [ ]:
import nltools
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain
from nilearn.maskers
import nibabel as nib

import os
import glob

import numpy as np
import pandas as pd
import statistics
import itertools
import random
import time

from scipy.stats import ttest_1samp
from scipy.stats.stats import pearsonr
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from nltools.stats import isc, fdr, threshold, phase_randomize, circle_shift, _butter_bandpass_filter, _phase_mean_angle, _phase_vector_length

import matplotlib.pyplot as plt
import seaborn as sns
from nilearn.plotting import view_img_on_surf, view_img, plot_surf_roi, plot_glass_brain, plot_stat_map

%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
data_dir = '/path/to/data/directory'

mask = Brain_Data('/path/to/mask/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii')
mask_x = expand_mask(mask)

mask.plot()

In [ ]:
sub_list = [os.path.basename(x).split('_')[0] for x in glob.glob(os.path.join(data_dir, 'fmriprep', '*', 'func', '*Part1*csv'))]
sub_list.sort()

# Whole-brain ISC

## 1. Gustave

In [ ]:
# Extract timepoints where Gustave (the protagonist) appears on screen

sub_list = [os.path.basename(x).split('_')[0] for x in glob.glob(os.path.join(data_dir,'fmriprep','*', 'func','*run-1*hdf5'))]
sub_list.sort()

pd.set_option('display.max_rows', None,'display.max_columns',None)

sub_timeseries = {}
sub_timeseries_gus={}

for sub in sub_list:
    run1 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-1_Average_ROI_schaefer100.csv'))
    run1 = run1.iloc[10:588]
    run2 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-2_Average_ROI_schaefer100.csv'))
    run2 = run2.iloc[20:488]
    run3 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-3_Average_ROI_schaefer100.csv'))
    run3 = run3.iloc[30:525]
    run4 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-4_Average_ROI_schaefer100.csv'))
    run4 = run4.iloc[22:608]
    run5 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-5_Average_ROI_schaefer100.csv'))
    run5 = run5.iloc[21:793]

    frame = [run1, run2, run3, run4, run5]

    sub_data = pd.DataFrame()

    for d in frame:
        sub_data = sub_data.append(d)
        
    sub_data.reset_index(inplace=True, drop=True)
        
    df1=sub_data.iloc[837:841]
    df2=sub_data.iloc[989:995]
    df3=sub_data.iloc[1159:1173]
    df4=sub_data.iloc[1518:1521]
    df5=sub_data.iloc[1525:1526]
    df6=sub_data.iloc[1528:1532]
    df7=sub_data.iloc[2040:2044]
    df8=sub_data.iloc[2050:2052]
    df9=sub_data.iloc[2054:2061]
    df10=sub_data.iloc[2062:2066]
    df11=sub_data.iloc[2068:2071]
    df12=sub_data.iloc[2072:2074]
    df13=sub_data.iloc[2603:2605]
    df14=sub_data.iloc[2606:2609]
    df15=sub_data.iloc[2640:2642]
    df16=sub_data.iloc[2669:2672]
    df17=sub_data.iloc[2675:2680]
    df18=sub_data.iloc[2686:2688]
    df19=sub_data.iloc[2690:2695]
    df20=sub_data.iloc[2699:2702]
    df21=sub_data.iloc[2707:2711]
    df22=sub_data.iloc[2714:2718]
    df23=sub_data.iloc[2723:2725]

    sub_data_gus = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, 
                               df12, df13, df14, df15, df16, df17, df18, df19, df20, 
                               df21, df22, df23])

    sub_timeseries_gus[sub] = sub_data_gus
    
sub_data_gus.head()

In [ ]:
roi = 67 # example ROI

def get_subject_roi(sub_data_gus, roi):
    sub_rois_gus = {}
    for sub in sub_data_gus:
        sub_rois_gus[sub] = sub_data_gus[sub].iloc[:, roi]
    return pd.DataFrame(sub_rois_gus)

sub_rois_gus = get_subject_roi(sub_timeseries_gus, roi)
sub_rois_gus.head()

In [ ]:
# Compute ISC during the appearance of Gustave for each ROI(region of interest)

isc_r_gus, isc_p_gus = {}, {}

for roi in range (100):
    stats = isc(get_subject_roi(sub_timeseries_gus, roi), n_bootstraps=5000, metric='median', method='bootstrap')
    isc_r_gus[roi], isc_p_gus[roi] = stats['isc'], stats['p']

In [ ]:
# Show the whole-brain ISC results

isc_val = pd.DataFrame.from_dict(isc_r_gus, orient = 'index', columns = ['ISC_gus'])
tmp = pd.DataFrame.from_dict(isc_p_gus, orient = 'index', columns = ['p_value'])
tmp.loc[tmp.p_value > .0005, 'p_value'] = 'ns'
isc_val = pd.concat([isc_val,tmp], axis = 1)
isc_val

## 2. Dmitri

In [ ]:
# Do the same for Dmitri, the antagonist

sub_timeseries_dmt={}

for sub in sub_list:
    run1 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-1_Average_ROI_schaefer100.csv'))
    run1 = run1.iloc[10:588]
    run2 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-2_Average_ROI_schaefer100.csv'))
    run2 = run2.iloc[20:488]
    run3 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-3_Average_ROI_schaefer100.csv'))
    run3 = run3.iloc[30:525]
    run4 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-4_Average_ROI_schaefer100.csv'))
    run4 = run4.iloc[22:608]
    run5 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-5_Average_ROI_schaefer100.csv'))
    run5 = run5.iloc[21:793]

    frame = [run1, run2, run3, run4, run5]

    sub_data = pd.DataFrame()

    for d in frame:
        sub_data = sub_data.append(d)
    sub_data.reset_index(inplace=True, drop=True)

    df1=sub_data.iloc[201:202]
    df2=sub_data.iloc[207:209]
    df3=sub_data.iloc[226:227]
    df4=sub_data.iloc[230:231]
    df5=sub_data.iloc[233:235]
    df6=sub_data.iloc[237:238]
    df7=sub_data.iloc[240:244]
    df8=sub_data.iloc[252:253]
    df9=sub_data.iloc[256:261]
    df10=sub_data.iloc[269:272]
    df11=sub_data.iloc[1419:1425]
    df12=sub_data.iloc[1428:1437]
    df13=sub_data.iloc[1439:1443]
    df14=sub_data.iloc[1448:1451]
    df15=sub_data.iloc[1455:1460]
    df16=sub_data.iloc[1463:1466]
    df17=sub_data.iloc[2233:2234]
    df18=sub_data.iloc[2249:2251]
    df19=sub_data.iloc[2266:2267]
    df20=sub_data.iloc[2329:2332]
    df21=sub_data.iloc[2333:2334]
    df22=sub_data.iloc[2349:2354]
    df23=sub_data.iloc[2358:2359]
    df24=sub_data.iloc[2360:2361]
    df25=sub_data.iloc[2365:2367]
    df26=sub_data.iloc[2374:2377]
    df27=sub_data.iloc[2378:2379]
    df28=sub_data.iloc[2387:2390]
    df29=sub_data.iloc[2393:2397]
    df30=sub_data.iloc[2399:2401]
    df31=sub_data.iloc[2405:2407]
    df32=sub_data.iloc[2409:2411]
    df33=sub_data.iloc[2417:2421]
    df34=sub_data.iloc[2436:2439]

    
    sub_data_dmt = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, 
                               df12, df13, df14, df15, df16, df17, df18, df19, df20, 
                               df21, df22, df23, df24, df25, df26, df27, df28, df29, 
                               df30, df31, df32, df33, df34])
    sub_timeseries_dmt[sub] = sub_data_dmt 

sub_data_dmt.head()

In [ ]:
def get_subject_roi_dmt(sub_data_dmt, roi):
    sub_rois_dmt = {}
    for sub in sub_data_dmt:
        sub_rois_dmt[sub] = sub_data_dmt[sub].iloc[:, roi]
    return pd.DataFrame(sub_rois_dmt)

sub_rois_dmt = get_subject_roi_dmt(sub_timeseries_dmt, roi)
sub_rois_dmt

In [ ]:
isc_r_dmt, isc_p_dmt = {}, {}

for roi in range (100):
    stats = isc(get_subject_roi(sub_timeseries_dmt, roi), n_bootstraps=5000, metric='median', method='bootstrap')
    isc_r_dmt[roi], isc_p_dmt[roi] = stats['isc'], stats['p']

In [ ]:
isc_val = pd.DataFrame.from_dict(isc_r_dmt, orient = 'index', columns = ['ISC_dmt'])
tmp = pd.DataFrame.from_dict(isc_p_dmt, orient = 'index', columns = ['p_value'])
tmp.loc[tmp.p_value > .0005, 'p_value'] = 'ns'
isc_val = pd.concat([isc_val,tmp], axis = 1)
isc_val

# Random sampling 1,000 iterations - Gustave, Dmitri

In [ ]:
iter_num = 1000
ran_TR = len(sub_data_gus)

sim = []
diff = []

sim_count = []
diff_count = []


for repeat in range (iter_num):
    ################### Dmitri ###################
    nums = random.sample(range(0,len(sub_data_dmt)),ran_TR) # randomly sample the timepoints where the antagonist appeared to match the number of TRs of the protagonist (who has less number of TRs)
    nums.sort()
    
    sub_list = [os.path.basename(x).split('_')[0] for x in glob.glob(os.path.join(data_dir,'fmriprep','*', 'func', '*run-1*hdf5'))]
    sub_list.sort()

    sub_timeseries_dmt={}
    for sub in sub_list:
        run1 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-1_Average_ROI_schaefer100.csv'))
        run1 = run1.iloc[10:588]
        run2 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-2_Average_ROI_schaefer100.csv'))
        run2 = run2.iloc[20:488]
        run3 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-3_Average_ROI_schaefer100.csv'))
        run3 = run3.iloc[30:525]
        run4 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-4_Average_ROI_schaefer100.csv'))
        run4 = run4.iloc[22:608]
        run5 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-5_Average_ROI_schaefer100.csv'))
        run5 = run5.iloc[21:793]

        frame = [run1, run2, run3, run4, run5]

        sub_data = pd.DataFrame()

        for d in frame:
            sub_data = sub_data.append(d)
        sub_data.reset_index(inplace=True, drop=True)

        df1=sub_data.iloc[201:202]
        df2=sub_data.iloc[207:209]
        df3=sub_data.iloc[226:227]
        df4=sub_data.iloc[230:231]
        df5=sub_data.iloc[233:235]
        df6=sub_data.iloc[237:238]
        df7=sub_data.iloc[240:244]
        df8=sub_data.iloc[252:253]
        df9=sub_data.iloc[256:261]
        df10=sub_data.iloc[269:272]
        df11=sub_data.iloc[1419:1425]
        df12=sub_data.iloc[1428:1437]
        df13=sub_data.iloc[1439:1443]
        df14=sub_data.iloc[1448:1451]
        df15=sub_data.iloc[1455:1460]
        df16=sub_data.iloc[1463:1466]
        df17=sub_data.iloc[2233:2234]
        df18=sub_data.iloc[2249:2251]
        df19=sub_data.iloc[2266:2267]
        df20=sub_data.iloc[2329:2332]
        df21=sub_data.iloc[2333:2334]
        df22=sub_data.iloc[2349:2354]
        df23=sub_data.iloc[2358:2359]
        df24=sub_data.iloc[2360:2361]
        df25=sub_data.iloc[2365:2367]
        df26=sub_data.iloc[2374:2377]
        df27=sub_data.iloc[2378:2379]
        df28=sub_data.iloc[2387:2390]
        df29=sub_data.iloc[2393:2397]
        df30=sub_data.iloc[2399:2401]
        df31=sub_data.iloc[2405:2407]
        df32=sub_data.iloc[2409:2411]
        df33=sub_data.iloc[2417:2421]
        df34=sub_data.iloc[2436:2439]

        df = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, 
                                   df12, df13, df14, df15, df16, df17, df18, df19, df20, 
                                   df21, df22, df23, df24, df25, df26, df27, df28, df29, 
                                   df30, df31, df32, df33, df34])

    
        sub_data_dmt=df.iloc[nums]
        sub_timeseries_dmt[sub] = sub_data_dmt
    print ('dataframe for Dmitri is ready...(',len(sub_data_dmt),"TRs)")
    
    #################### Gustave ##################
    
    sub_timeseries_gus={}
    
    for sub in sub_list:
        run1 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-1_Average_ROI_schaefer100.csv'))
        run1 = run1.iloc[10:588]
        run2 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-2_Average_ROI_schaefer100.csv'))
        run2 = run2.iloc[20:488]
        run3 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-3_Average_ROI_schaefer100.csv'))
        run3 = run3.iloc[30:525]
        run4 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-4_Average_ROI_schaefer100.csv'))
        run4 = run4.iloc[22:608]
        run5 = pd.read_csv(os.path.join(data_dir, 'fmriprep', sub, 'func', f'{sub}_run-5_Average_ROI_schaefer100.csv'))
        run5 = run5.iloc[21:793]

        frame = [run1, run2, run3, run4, run5]

        sub_data = pd.DataFrame()

        for d in frame:
            sub_data = sub_data.append(d)

        sub_data.reset_index(inplace=True, drop=True)

        df1=sub_data.iloc[837:841]
        df2=sub_data.iloc[989:995]
        df3=sub_data.iloc[1159:1173]
        df4=sub_data.iloc[1518:1521]
        df5=sub_data.iloc[1525:1526]
        df6=sub_data.iloc[1528:1532]
        df7=sub_data.iloc[2040:2044]
        df8=sub_data.iloc[2050:2052]
        df9=sub_data.iloc[2054:2061]
        df10=sub_data.iloc[2062:2066]
        df11=sub_data.iloc[2068:2071]
        df12=sub_data.iloc[2072:2074]
        df13=sub_data.iloc[2603:2605]
        df14=sub_data.iloc[2606:2609]
        df15=sub_data.iloc[2640:2642]
        df16=sub_data.iloc[2669:2672]
        df17=sub_data.iloc[2675:2680]
        df18=sub_data.iloc[2686:2688]
        df19=sub_data.iloc[2690:2695]
        df20=sub_data.iloc[2699:2702]
        df21=sub_data.iloc[2707:2711]
        df22=sub_data.iloc[2714:2718]
        df23=sub_data.iloc[2723:2725]

        sub_data_gus = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, 
                                   df12, df13, df14, df15, df16, df17, df18, df19, df20, 
                                   df21, df22, df23])



        sub_timeseries_gus[sub] = sub_data_gus      
    
    def get_subject_roi_gus (sub_data_gus, roi):
        sub_rois_gus = {}
        for sub in sub_data_gus:
            sub_rois_gus[sub] = sub_data_gus[sub].iloc[:, roi]
        return pd.DataFrame(sub_rois_gus)

    sub_rois_gus = get_subject_roi_gus(sub_timeseries_gus, roi)
    ##

    isc_r_gus, isc_p_gus = {}, {}
    for roi in range(100):
        stats_gus = isc(get_subject_roi_gus(sub_timeseries_gus, roi), n_bootstraps=5000, metric='median', method='bootstrap')
        isc_r_gus[roi], isc_p_gus[roi] = stats_gus['isc'], stats_gus['p']

    isc_r_gus_brain, isc_p_gus_brain = roi_to_brain(pd.Series(isc_r_gus), mask_x), roi_to_brain(pd.Series(isc_p_gus), mask_x)


    def get_subject_roi_dmt(sub_data_dmt, roi):
        sub_rois_dmt = {}
        for sub in sub_data_dmt:
            sub_rois_dmt[sub] = sub_data_dmt[sub].iloc[:, roi]
        return pd.DataFrame(sub_rois_dmt)
    
    sub_rois_dmt = get_subject_roi_dmt(sub_timeseries_dmt, roi)
    
    ##
    
    isc_r_dmt, isc_p_dmt = {}, {}
    for roi in range(100):
        stats_dmt = isc(get_subject_roi_dmt(sub_timeseries_dmt, roi), n_bootstraps=5000, metric='median', method='bootstrap')
        isc_r_dmt[roi], isc_p_dmt[roi] = stats_dmt['isc'], stats_dmt['p']

    isc_r_dmt_brain, isc_p_dmt_brain = roi_to_brain(pd.Series(isc_r_dmt), mask_x), roi_to_brain(pd.Series(isc_p_dmt), mask_x)

    col_name = []
    for i in range(100):
        col_name.append("Gustave_roi_" + str(i))

    pair_list_gus = [pd.DataFrame()]

    for roi in range(100):

        sub_rois_gus = get_subject_roi_gus(sub_timeseries_gus, roi)
        sub_rois_gus

        correlations = {}
        columns = sub_rois_gus.columns.tolist()

        for col_a, col_b in itertools.combinations(columns, 2):
            correlations[col_a + '__' + col_b] = pearsonr(sub_rois_gus.loc[:, col_a], sub_rois_gus.loc[:, col_b])

        result = DataFrame.from_dict(correlations, orient='index')
        result.columns = ['PCC', 'p-value']
        corr_result = DataFrame.from_dict(correlations, orient='index')

        corr_list = corr_result.iloc[:,0].values.tolist()
        corr_list = DataFrame.from_dict(corr_list)
        pair_list_gus.append(corr_list)


    pair_list_gus = pd.concat(pair_list_gus, axis = 1)
    pair_list_gus.columns = col_name
    
    ##
    
    col_name = []
    for i in range(100):
        col_name.append("Dmitri_roi_" + str(i))

    pair_list_dmt = [pd.DataFrame()]

    for roi in range(100):

        sub_rois_dmt = get_subject_roi_dmt(sub_timeseries_dmt, roi)

        correlations = {}
        columns = sub_rois_dmt.columns.tolist()

        for col_a, col_b in itertools.combinations(columns, 2):
            correlations[col_a + '__' + col_b] = pearsonr(sub_rois_dmt.loc[:, col_a], sub_rois_dmt.loc[:, col_b])

        result = DataFrame.from_dict(correlations, orient='index')
        result.columns = ['PCC', 'p-value']

        corr_result = DataFrame.from_dict(correlations, orient='index')

        corr_list = corr_result.iloc[:,0].values.tolist()
        corr_list = DataFrame.from_dict(corr_list)
        pair_list_dmt.append(corr_list)

    pair_list_dmt = pd.concat(pair_list_dmt, axis = 1)
    pair_list_dmt.columns = col_name

    for roi in range (100):

        gus = np.arctanh(pair_list_gus.iloc[:,roi])
        dmt = np.arctanh(pair_list_dmt.iloc[:,roi])

        t_test = stats.ttest_rel(gus, dmt)

        if t_test[1] < 0.0005:
            diff.append(str(roi))
        else :
            sim.append(str(roi))
    
diff_list = []
for roi in range (100):
    tmp = different.count(str(roi))
    diff_list.append(tmp)

sim_list = []
for roi in range (100):
    tmp = similar.count(str(roi))
    sim_list.append(tmp)

sim_count = pd.DataFrame(sim_list, columns=['Not Different'])
diff_count = pd.DataFrame(diff_list, columns = ['Different'])
                       
iter_count = pd.concat([sim_count, diff_count],axis=1)
print(iter_count) # The results can be shown as Figure 4 lower panel